In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision
from utils.transform import VideoTransform
from utils.dataset import UCF101Dataset

traindataset = UCF101Dataset(
    root='./data/UCF-101',
    annotation_path='./data/UCF_annotations',
    frames_per_clip=16,
    train=True,
    num_workers=8,
    transform=VideoTransform(resize=(224, 224), phase='train')
    )


valdataset = UCF101Dataset(
    root='./data/UCF-101',
    annotation_path='./data/UCF_annotations',
    frames_per_clip=16,
    train=False,
    num_workers=8,
    transform=VideoTransform(resize=(224, 224), phase='val')
)

print(traindataset.__len__())
print(valdataset.__len__())

/home/vlsimin95/anaconda3/envs/torch38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 833/833 [00:36<00:00, 22.66it/s]


1643024
640471


In [2]:
train_dataloader = DataLoader(traindataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(valdataset, batch_size=16, shuffle=True)

In [3]:
device = 'cpu' if torch.cuda.is_available() else 'cpu'
print(f'using {device} device')

using cpu device


In [4]:
from model.eco_lite import ECO_Lite
model = ECO_Lite().to(device)
print(model)

ECO_Lite(
  (eco_2d): ECO_2D(
    (basic_conv): BasicConv(
      (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1_relu_7x7): ReLU(inplace=True)
      (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3_reduce): ReLU(inplace=True)
      (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3): ReLU(inplace=True)
      (pool2_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (inception_a): InceptionA(
      (inc

In [5]:
# model test
X = torch.rand(1, 16, 3, 224, 224, device=device)
logits = model(X)
print(logits.shape)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predictied Class(Random input) : {y_pred}')

torch.Size([1, 101])
Predictied Class(Random input) : tensor([30])


In [6]:
epochs = 5
learning_rate = 1e-5

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate, momentum=0.9, dampening=0, nesterov=True)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [9]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(val_dataloader, model, loss_fn)

Epoch 1
-------------------------------


/home/vlsimin95/anaconda3/envs/torch38/lib/python3.8/site-packages/torchvision/io/video.py:162: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


loss: 4.647573  [    0/1643024]


KeyboardInterrupt: 